In [8]:
import numpy as np
import pandas as pd
from datetime import datetime
import networkx as nx
from scipy.special import softmax
from tqdm import tqdm
from random import shuffle
import copy
from collections import defaultdict
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt

In [9]:
data = pd.read_csv('nyc_citi.csv')
data = data.loc[[type(i)==str for i in data['start station name']]]
data = data.loc[[type(i)==str for i in data['end station name']]]
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,695,2013-06-01 00:00:01,2013-06-01 00:11:36,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,19678,Subscriber,1983.0,1
1,693,2013-06-01 00:00:08,2013-06-01 00:11:41,444,Broadway & W 24 St,40.742354,-73.989151,434.0,9 Ave & W 18 St,40.743174,-74.003664,16649,Subscriber,1984.0,1
2,2059,2013-06-01 00:00:44,2013-06-01 00:35:03,406,Hicks St & Montague St,40.695128,-73.995951,406.0,Hicks St & Montague St,40.695128,-73.995951,19599,Customer,NaN,0
3,123,2013-06-01 00:01:04,2013-06-01 00:03:07,475,E 15 St & Irving Pl,40.735243,-73.987586,262.0,Washington Park,40.691782,-73.973730,16352,Subscriber,1960.0,1
4,1521,2013-06-01 00:01:22,2013-06-01 00:26:43,2008,Little West St & 1 Pl,40.705693,-74.016777,310.0,State St & Smith St,40.689269,-73.989129,15567,Subscriber,1983.0,1


In [10]:
data.starttime=data.starttime.apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
data.stoptime=data.stoptime.apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
print(data.shape)

(559644, 15)


In [13]:
def euc_dis(x1,y1,x2,y2):
    return ((x1-x2)**2+(y1-y2)**2)**.5

In [14]:
def build_station_graph(data,starttime,stoptime):
    graphx=nx.Graph()
    filtered_data = data[(data['starttime']>=starttime) & (data['stoptime']<=stoptime)]
    locations = list(set(filtered_data['start station name']).union(filtered_data['end station name']))
    vertices = pd.Series([{}]*len(locations),index=locations)
    for loc in locations:
        vertices[loc] = {'change':0,'x':np.random.random(),'y':np.random.random(),'type':''}
        
    for i in filtered_data['start station name']:
        vertices[i]['change'] -= 1
    for i in filtered_data['end station name']:
        vertices[i]['change'] += 1
    for loc in locations:
        if vertices[loc]['change'] > 0:
            vertices[loc]['type'] = 'overflow'
        if vertices[loc]['change'] < 0:
            vertices[loc]['type'] = 'underflow'
    vertices = vertices.loc[[x['type'] != '' for x in vertices]]
    for vertex in vertices.index:
        graphx.add_node(vertex,bipartite=int(vertices[vertex]['type']=='overflow'))
    #print(vertices)
    nx.set_node_attributes(graphx, {i: vertices[i] for i in vertices.index}) 
    for i in range(len(vertices.index)-1):
        for j in range(len(vertices.index[i:])):
            if vertices[vertices.index[i]]['type'] != vertices[vertices.index[j]]['type']:
                x1=vertices[vertices.index[i]]['x']
                y1=vertices[vertices.index[i]]['y']
                x2=vertices[vertices.index[j]]['x']
                y2=vertices[vertices.index[j]]['y']
                graphx.add_weighted_edges_from([(vertices.index[i],vertices.index[j],euc_dis(x1,y1,x2,y2))])
    return graphx

def cloned_station_vertices(graphx):
    nodes = list(graphx.nodes())
    for node in nodes:
        if graphx.nodes[node]['change']>1:
            for i in range(graphx.nodes[node]['change']-1):
                i = i+1
                graphx.add_nodes_from([(str(node)+str(i),graphx.nodes[node])],bipartite=int(graphx.nodes[node]['type']=='underflow'))
                #print(graphx[str(node)+str(i)])
                new_node_name = str(node)+str(i)
                new_node_data = graphx.nodes[new_node_name]
                for node_ in graphx.nodes:
                    if np.sign(new_node_data['change']) != np.sign(graphx.nodes[node_]['change']):
                        x1=new_node_data['x']
                        y1=new_node_data['y']
                        x2=graphx.nodes[node_]['x']
                        y2=graphx.nodes[node_]['y']
                        graphx.add_weighted_edges_from([(node_,new_node_name,euc_dis(x1,y1,x2,y2))])
    return graphx
                


                    
            

In [16]:
graph = build_station_graph(data,data.loc[0,'starttime'],data.loc[10,'stoptime'])
#print(graph['E 6 St & Avenue D'])
overflow = {node:{} for node in graph.nodes() if graph.nodes[node]['type']=='overflow'}
underflow = {node:{} for node in graph.nodes() if graph.nodes[node]['type']=='underflow'}


cgraph = cloned_station_vertices(graph)
workers = {i:{
                'type':'worker',
               'xs':np.random.random(),
               'ys':np.random.random(),
               'xe':np.random.random(),
               'ye':np.random.random()
              } for i in range(np.min([len(underflow), len(overflow)]))}
cwgraph = cgraph.copy()
print(cwgraph.nodes)
cwgraph.add_nodes_from(range(len(workers)))
nx.set_node_attributes(cwgraph, workers) 
cwgraph.add_edges_from([(w,s) for w in workers for s in cgraph.nodes ])
print(cwgraph.nodes)
print(cwgraph.nodes[1])

['DeKalb Ave & S Portland Ave', 'W 13 St & 7 Ave', 'W 54 St & 9 Ave', 'E 58 St & 3 Ave', '10 Ave & W 28 St', 'Cadman Plaza West & Montague St', 'Atlantic Ave & Fort Greene Pl', '8 Ave & W 31 St N', 'Shevchenko Pl & E 7 St', 'Washington Park', 'Ashland Pl & Hanson Pl', 'W 37 St & 5 Ave', 'Pershing Square South', 'Fulton St & Grand Ave', 'Murray St & West St', 'Catherine St & Monroe St', 'E 10 St & Avenue A', 'Allen St & Rivington St', 'W 37 St & 10 Ave', 'Lafayette St & E 8 St', 'Forsyth St & Broome St', 'Suffolk St & Stanton St', 'Broadway & W 24 St', 'W 22 St & 8 Ave', 'Mercer St & Spring St', 'E 20 St & 2 Ave', 'Cleveland Pl & Spring St', 'W 22 St & 10 Ave', '9 Ave & W 18 St', '9 Ave & W 22 St', 'Greenwich St & W Houston St', 'University Pl & E 14 St', 'E 31 St & 3 Ave', 'E 6 St & Avenue B', 'St Marks Pl & 2 Ave', 'Pike St & E Broadway', 'Clermont Ave & Lafayette Ave', 'W 26 St & 8 Ave', 'E 4 St & 2 Ave', '9 Ave & W 14 St', 'Allen St & Stanton St', 'Pershing Square North', 'Christoph

In [18]:

class UGA:
    def __init__(self,meta_graph):
        self.meta_graph = meta_graph
        self.overflow = [node for node in meta_graph.nodes() if meta_graph.nodes[node]['type'] == 'overflow']
        self.worker = [node for node in meta_graph.nodes() if meta_graph.nodes[node]['type'] == 'worker']
        self.underflow = [node for node in meta_graph.nodes() if meta_graph.nodes[node]['type'] == 'underflow']
        
    def add_triplet(self, vertex, graph):
        neighbors = list(self.meta_graph.neighbors(vertex))
        shuffle(neighbors)
        overflow = None
        underflow = None
        for overflow_ in neighbors:
            if (not overflow_ in list(graph.nodes())) and self.meta_graph.nodes[overflow_]['type']=='overflow':
                
                overflow = overflow_
        for underflow_ in neighbors:
            if (not underflow_ in list(graph.nodes())) and self.meta_graph.nodes[underflow_]['type']=='underflow':
                underflow = underflow_
        if overflow != None and underflow != None:    
            graph.add_node(overflow)
            graph.add_node(underflow)
            graph.add_node(vertex)
            graph.add_edge(vertex,underflow)
            graph.add_edge(vertex,overflow)
            
    
            
    #each species is a complete of worker station triples
    def build_matching(self):
        graph = nx.Graph()
        
        shuffle(self.worker)
        for w in self.worker:
            self.add_triplet(w,graph)
        graph.add_nodes_from([i for i in self.meta_graph.nodes if not i in list(graph.nodes)])
        return graph
    
    def find_overflow(self,vertex):
        while True:
            node_ind = np.random.choice(range(len(self.overflow)))
            if self.overflow[node_ind] != vertex:
                return self.overflow[node_ind]
            
    def find_underflow(self,vertex):
        while True:
            node_ind = np.random.choice(range(len(self.underflow)))
            if self.underflow[node_ind] != vertex:
                return self.underflow[node_ind]
            
    
    def swap(self, worker, to_replace, graph):
        if self.meta_graph.nodes[to_replace]['type'] == 'overflow':
            other = self.find_overflow(to_replace)
        else:
            other = self.find_underflow(to_replace)
        if len(list(graph.neighbors(other)))>0:
            otherw = list(graph.neighbors(other))[0]
            
            graph.add_edge(otherw,to_replace)
            graph.remove_edge(otherw,other)
        graph.add_edge(worker,other)
        graph.remove_edge(worker,to_replace)
            
        
        
    def euc_fitness(self,spec):
        return sum([self.euc_fitness_ind(gene,spec) for gene in spec.nodes if self.meta_graph.nodes[gene]['type'] == 'worker'])

    def euc_fitness_ind(self,gene,spec):
        w = 0


        neighbors = list(spec.neighbors(gene))
        if len(neighbors) != 0:
            for i in neighbors:
                if self.meta_graph.nodes[i]['type'] == 'underflow':
                    w += euc_dis(self.meta_graph.nodes[i]['x'], self.meta_graph.nodes[i]['y'],
                                 self.meta_graph.nodes[gene]['xs'],self.meta_graph.nodes[gene]['ys'])
                else:
                    w += euc_dis(self.meta_graph.nodes[i]['x'],self.meta_graph.nodes[i]['y'],
                                 self.meta_graph.nodes[gene]['xe'],self.meta_graph.nodes[gene]['ye'])
            w += euc_dis(self.meta_graph.nodes[neighbors[0]]['x'],self.meta_graph.nodes[neighbors[0]]['y'],
                         self.meta_graph.nodes[neighbors[1]]['x'],self.meta_graph.nodes[neighbors[1]]['y'])

        return -w

    def compare_specs(self,s1,s2,fitness):
        return self.euc_fitness(s1) > self.euc_fitness(s2)

    
            
    def run(self, sswap_rate, oswap_rate, gens, pop_size):
        #make a population
        pop = [self.build_matching() for i in range(pop_size)]
        
        bests = []
        for gen in tqdm(range(gens)):
            #get scores of species
            scores = np.array([self.euc_fitness(s) for s in pop])
            bests.append(max(scores))
            
            scores = softmax(scores)  
            print(scores[:20])
            selected = []
            
            #while selecting pick a species
            while len(selected) < len(pop):
                spec_ind = np.random.choice(list(range(len(pop))), p=scores)
                spec = pop[spec_ind]
                #for each gene (a triple)
                for j,node in enumerate(spec.nodes):
                    #swap with another pop
                    edges = set(spec.edges)
                    if self.meta_graph.nodes[node]['type']!= 'worker' and len(list(spec.neighbors(node)))>0 and np.random.random() < sswap_rate:
                        self.swap(list(spec.neighbors(node))[0], node, spec)
                        print(edges.intersection(set(self.meta_graph.edges).intersection(set(spec.edges))))
                selected.append(spec)
                                    
            pop = selected
        return pop[np.argmax(scores)],np.max(scores), bests
                
                
    
                            



In [19]:
uga = UGA(cwgraph)

best, score, log = uga.run(0.5,0.001,400,100)

, 14), ('University Pl & E 14 St', 19), ('W 37 St & 10 Ave', 6), ('Lafayette St & E 8 St', 10), ('E 58 St & 3 Ave', 8), ('E 10 St & Avenue A', 3), ('8 Ave & W 31 St N1', 16), ('Broadway & W 24 St', 2), ('E 20 St & 2 Ave', 17), ('Broadway & W 29 St', 4), ('Mercer St & Spring St', 8), ('Catherine St & Monroe St', 13)}
{('9 Ave & W 22 St', 21), ('10 Ave & W 28 St', 15), ('9 Ave & W 18 St1', 10), ('8 Ave & W 31 St N1', 16), ('E 10 St & Avenue A', 3), ('W 54 St & 9 Ave', 14), ('Murray St & West St', 7), ('Broadway & W 24 St', 2), ('University Pl & E 14 St', 19), ('Lafayette St & E 8 St', 10), ('E 20 St & 2 Ave', 17), ('Broadway & W 29 St', 4), ('E 58 St & 3 Ave', 8), ('Forsyth St & Broome St', 19), ('W 37 St & 10 Ave', 6), ('Mercer St & Spring St', 8), ('Cadman Plaza West & Montague St', 0), ('Catherine St & Monroe St', 13)}
{('9 Ave & W 22 St', 21), ('10 Ave & W 28 St', 15), ('9 Ave & W 18 St1', 10), ('8 Ave & W 31 St N1', 16), ('E 10 St & Avenue A', 3), ('Murray St & West St', 7), ('Broad

KeyboardInterrupt: 

In [ ]:
plt.plot(log)
plt.show()

In [ ]:
len(overflow)

In [ ]:

edge_list = defaultdict(lambda:{})
for edge in cgraph.edges:
    if graph.edges[edge]['weight'] > 0 and cgraph.nodes[edge[0]]['type'] != 'worker' and cgraph.nodes[edge[0]]['type'] != 'worker':
        edge_list[edge[0]][edge[1]] = graph.edges[edge]['weight']
        edge_list[edge[1]][edge[0]] = graph.edges[edge]['weight']
#print(edge_list)

worker = {node:{} for node in cwgraph.nodes() if cwgraph.nodes[node]['type']=='worker'}



In [ ]:

matching = bipartite.matching.minimum_weight_full_matching(cgraph)
spwgraph =nx.Graph()
for key in matching.keys():
    if cgraph.nodes[key]['type'] =='overflow':
        for w, worker_ in enumerate(worker):
            spwgraph.add_edge((key+'|'+matching[key]),w,
                              weight=cgraph.edges[key, matching[key]]['weight']+
                             euc_dis(cwgraph.nodes[worker_]['xs'],cwgraph.nodes[worker_]['ys'],cgraph.nodes[matching[key]]['x'],cgraph.nodes[matching[key]]['y']) +
                             euc_dis(cwgraph.nodes[worker_]['xe'],cwgraph.nodes[worker_]['ye'],cgraph.nodes[matching[key]]['x'],cgraph.nodes[matching[key]]['y'])
                                         )
matching2 = bipartite.matching.minimum_weight_full_matching(spwgraph)
matching2


In [ ]:
print(graph.nodes['Maodison St & Montgomery St'])

In [ ]:
cgraph.edges['Rivington St & Chrystie St', 'Rivington St & Chrystie St']